In [1]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>
# To horizontal scroll dataframe table

In [152]:
import graphframes
from pyspark.sql import SparkSession
import os

# for pre-processing
from pyspark.sql.functions import col
from pyspark.sql.functions import exp
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import min, max

# for scaling
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler # transform to vector format before scaling
from pyspark.sql.functions import udf #
from pyspark.sql.types import DoubleType

# for reputation score calculation
from graphframes import GraphFrame

# for post procesing
from pyspark.sql.functions import sum as _sum # avoid same name with sum from python


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Function to calculating weight of the edge(txn) by the age of the txn
def logistic(x,
             a=2,
             b=1 / 64):
    return a / (1.0 + np.exp(-b * x))
def normalize(df):
    normalized_df=(df-df.min())/(df.max()-df.min())
    return normalized_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession \
    .builder \
    .config("fs.s3a.access.key", "***") \
    .config("fs.s3a.secret.key", "***") \
    .appName("Test") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 1. Data Preprocessing

In [50]:
#''' # get all file

bucket="s3a://octan-labs-bsc/"
path="/transactions/"
data_df=spark.read.format("csv") \
    .option("header", True) \
    .load(bucket+path+'*/*/*.csv')

''' test one file

bucket="s3a://octan-labs-bsc/"
path="/transactions/start_block=0000420000/"
data_df=spark.read.format("csv") \
    .option("header", True) \
    .load(bucket+path+'*/*.csv')
    
#'''

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

' test one file\n\nbucket="s3a://octan-labs-bsc/"\npath="/transactions/start_block=0000420000/"\ndata_df=spark.read.format("csv")     .option("header", True)     .load(bucket+path+\'*/*.csv\')\n\n#'

In [51]:
data_df = data_df.select(col('block_number'),col('from_address'),col('to_address'),col('gas'),col('gas_price'))
data_df.show()#truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+--------------------+-------------------+-----------+
|block_number|        from_address|          to_address|                gas|  gas_price|
+------------+--------------------+--------------------+-------------------+-----------+
|      400200|0xcc55aa8fa576a02...|0x2170ed0880ac9a7...|              54190|91000000000|
|      400200|0x2ecce7198df1180...|0x75ca8f6c82df5fd...|             992980|20000000000|
|      400200|0x2f7be8361c80a4c...|0x000000000000000...|9223372036854775807|          0|
|      400200|0x2f7be8361c80a4c...|0x000000000000000...|9223372036854775807|          0|
|      400201|0xb005741528b86f5...|0x000000000000000...|            4700000|20000000000|
|      400201|0xb005741528b86f5...|0x000000000000000...|            4700000|20000000000|
|      400201|0xb730d3908b9b83a...|0xae9269f27437f0f...|              66006|20000000000|
|      400201|0x35e7a025f4da968...|0x000000000000000...|9223372036854775807|          0|
|      400201|0x35e7a

#### Summary before processing

In [52]:
print('Total: ',data_df.count())
block_min = data_df.select(min('block_number')).first()[0]
block_max = data_df.select(max('block_number')).first()[0]
print('first block:',block_min,'last block:',block_max)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total:  234853
first block: 1 last block: 99879

#### Get the Used gas

In [53]:
data_df = data_df.withColumn('gas_used (bnb)', col('gas') * col('gas_price'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
data_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+--------------------+-------------------+-----------+--------------+
|block_number|        from_address|          to_address|                gas|  gas_price|gas_used (bnb)|
+------------+--------------------+--------------------+-------------------+-----------+--------------+
|      400200|0xcc55aa8fa576a02...|0x2170ed0880ac9a7...|              54190|91000000000|    4.93129E15|
|      400200|0x2ecce7198df1180...|0x75ca8f6c82df5fd...|             992980|20000000000|    1.98596E16|
|      400200|0x2f7be8361c80a4c...|0x000000000000000...|9223372036854775807|          0|           0.0|
|      400200|0x2f7be8361c80a4c...|0x000000000000000...|9223372036854775807|          0|           0.0|
|      400201|0xb005741528b86f5...|0x000000000000000...|            4700000|20000000000|        9.4E16|
|      400201|0xb005741528b86f5...|0x000000000000000...|            4700000|20000000000|        9.4E16|
|      400201|0xb730d3908b9b83a...|0xae9269f27437f0f...|        

### remove loop, keep last txn, remove nan and null address

In [55]:
 # remove loop
data_df = data_df.filter(col('from_address') != col('to_address'))
 # keep last txn
window_spec = Window.partitionBy(['from_address', 'to_address']).orderBy(col('block_number').desc())
data_df = data_df.withColumn('row_num', row_number().over(window_spec)).filter(col('row_num') == 1).drop('row_num')
# remove nan
data_df = data_df.dropna(subset=['from_address'])
data_df = data_df.dropna(subset=['to_address'])
# remove null address (address for burning)
data_df = data_df.filter(col('from_address') != '0x0000000000000000000000000000000000000000') 
data_df = data_df.filter(col('from_address') != '0x000000000000000000000000000000000000dEaD') 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Example:
Before pre-processing

| block_num | from_address | to_address |
| --- | --- | --- |
| 1 | a | b |
| 1 | a | b |
| 2 | a | b |
| 2 | a | null |
| 3 | a |  |
| 3 | a | a |

After pre-processing

| block_num | from_address | to_address |
| --- | --- | --- |
| 2 | a | b |


#### Summary after pre-processing

In [56]:
print('Total:', data_df.count(), 'txns')
block_min = data_df.select(min('block_number')).first()[0]
block_max = data_df.select(max('block_number')).first()[0]
print('first block:',block_min,'last block:',block_max)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total: 19154 txns
first block: 1 last block: 99879

## 2. Calculting weight for node and edge

In this verion, graphframe will be used to calculate page rank. However, due to limitation of this package, it cannot assign weight to edges and node. </br>
Therefore, graphframe part will be used as a place holder for better package. </br>
Weight for node and edge are still calculated but will not be used.

In [57]:
activate_time=90
blockperday_set=28800
data_df = data_df.withColumn('block_date', ((col('block_number') - block_min - activate_time) / blockperday_set).cast('int'))
data_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+--------------------+------+------------+--------------+----------+
|block_number|        from_address|          to_address|   gas|   gas_price|gas_used (bnb)|block_date|
+------------+--------------------+--------------------+------+------------+--------------+----------+
|      402026|0x000000004fa9e63...|0x8f76c99c3e96371...| 21000| 91000000000|      1.911E15|        13|
|      402822|0x000819e432ac8eb...|0xae9269f27437f0f...| 66006| 20000000000|    1.32012E15|        13|
|      402702|0x000819e432ac8eb...|0xe9e7cea3dedca59...| 66213| 20000000000|    1.32426E15|        13|
|      415290|0x00197a17c8ec9e0...|0x0d76db4e067be7f...|152910| 20000000000|     3.0582E15|        14|
|      420302|0x00197a17c8ec9e0...|0x2170ed0880ac9a7...| 76708|103000000000|   7.900924E15|        14|
|      394931|0x00197a17c8ec9e0...|0x87d22a430356e8d...|152926| 20000000000|    3.05852E15|        13|
|      399545|0x00197a17c8ec9e0...|0x89da130d26b6dba...|152926| 200000000

In [58]:
a = 1
b = 1/64
data_df = data_df.withColumn('age_weight', a / (1.0 + exp(-b * data_df['block_date'])))
data_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+--------------------+------+------------+--------------+----------+------------------+
|block_number|        from_address|          to_address|   gas|   gas_price|gas_used (bnb)|block_date|        age_weight|
+------------+--------------------+--------------------+------+------------+--------------+----------+------------------+
|      402026|0x000000004fa9e63...|0x8f76c99c3e96371...| 21000| 91000000000|      1.911E15|        13|0.5506073655352404|
|      402822|0x000819e432ac8eb...|0xae9269f27437f0f...| 66006| 20000000000|    1.32012E15|        13|0.5506073655352404|
|      402702|0x000819e432ac8eb...|0xe9e7cea3dedca59...| 66213| 20000000000|    1.32426E15|        13|0.5506073655352404|
|      415290|0x00197a17c8ec9e0...|0x0d76db4e067be7f...|152910| 20000000000|     3.0582E15|        14|0.5544704649604273|
|      420302|0x00197a17c8ec9e0...|0x2170ed0880ac9a7...| 76708|103000000000|   7.900924E15|        14|0.5544704649604273|
|      394931|0x00197a17

### Get weight for edge

In [59]:
to_weights_df = data_df.select(['to_address', 'age_weight'])
node_age_weights = to_weights_df.groupBy('to_address').agg(_sum('age_weight').alias('age_weight'))
s_weights = node_age_weights.agg(_sum('age_weight')).collect()[0][0] # result is a list of row so need [0][0]
personalized_weights = {row['to_address']: row['age_weight']/s_weights for row in node_age_weights.collect()}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
data_df.show()#truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+--------------------+------+------------+--------------+----------+------------------+
|block_number|        from_address|          to_address|   gas|   gas_price|gas_used (bnb)|block_date|        age_weight|
+------------+--------------------+--------------------+------+------------+--------------+----------+------------------+
|      402026|0x000000004fa9e63...|0x8f76c99c3e96371...| 21000| 91000000000|      1.911E15|        13|0.5506073655352404|
|      402822|0x000819e432ac8eb...|0xae9269f27437f0f...| 66006| 20000000000|    1.32012E15|        13|0.5506073655352404|
|      402702|0x000819e432ac8eb...|0xe9e7cea3dedca59...| 66213| 20000000000|    1.32426E15|        13|0.5506073655352404|
|      415290|0x00197a17c8ec9e0...|0x0d76db4e067be7f...|152910| 20000000000|     3.0582E15|        14|0.5544704649604273|
|      420302|0x00197a17c8ec9e0...|0x2170ed0880ac9a7...| 76708|103000000000|   7.900924E15|        14|0.5544704649604273|
|      394931|0x00197a17

### Get weight for node(vertices)

In [61]:
vertices_df = data_df.select(col("from_address").alias("id")).union(data_df.select(col("to_address").alias("id")))
vertices_df = vertices_df.distinct()
vertices_df.show()#truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|                  id|
+--------------------+
|0x5558bf3dc2f5b0c...|
|0x56e5272a583d3b4...|
|0x96adc9b48983b02...|
|0xb0dd17567933fc8...|
|0xc8b9bbba82cbe9f...|
|0xd2f93484f2d3191...|
|0xd576b5994865c94...|
|0x0e630dc7ba2de34...|
|0x1c537fcf80c1069...|
|0x4e8f1cf9a9ddcad...|
|0x54b4e4a37808b8d...|
|0x555a56c3f71d8d1...|
|0x67909f0fd4d7634...|
|0x70ddb0f995d235e...|
|0x83003d1b9d4e472...|
|0x9e8b8a310be07b8...|
|0xecb0a819c0d9914...|
|0xf764cc532f61789...|
|0x0cae1e6c5e8b410...|
|0x15222740c04c23a...|
+--------------------+
only showing top 20 rows

In [62]:
weight_df = spark.createDataFrame(
    personalized_weights.items(),
    ["id", "weight"]
)
result_df = vertices_df.join(weight_df, "id", "left").select("id", "weight")
vertices_df_with_personalized_weight = result_df.withColumnRenamed("weight", "personalized weight")
vertices_df_with_personalized_weight=vertices_df_with_personalized_weight.fillna(0)
vertices_df_with_personalized_weight.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------+---------------------+
|id                                        |personalized weight  |
+------------------------------------------+---------------------+
|0x56e5272a583d3b490062f69e183d4a510d007ef8|5.2017130048517535E-5|
|0xb0dd17567933fc80810d7fd6b39409eb03dd23a6|1.0476417167384429E-4|
|0xc8b9bbba82cbe9f56ea20344cd5dc84066f994ed|5.2017130048517535E-5|
|0xd576b5994865c94a0a6449687c2a08e47fd77bb4|5.2017130048517535E-5|
|0x0e630dc7ba2de34887ac7173f9d480070c810526|5.2382085836922145E-5|
|0x1c537fcf80c1069716ea7a151100edbd55b99a4c|1.0476417167384429E-4|
|0x4e8f1cf9a9ddcadee3340dada2cba5508d340b4e|5.2017130048517535E-5|
|0x555a56c3f71d8d1c411bd3b10434385c411547fe|1.0439921588543968E-4|
|0x67909f0fd4d76342b85108412d5c97ed5b33c49f|5.2382085836922145E-5|
|0xf764cc532f61789b0cf56dab37348f02afc70887|5.2382085836922145E-5|
|0xfa97c22a03d8522988c709c24283c0918a59c795|5.2382085836922145E-5|
|0x0cae1e6c5e8b4109097e9e900e9f9554cc26b66c|5.2382085836922145

In [63]:
edges_df = data_df.select(col("from_address").alias("src"), col("to_address").alias("dst"), col("age_weight").alias("weight"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
print(vertices_df.count(),edges_df.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4924 19154

### Get transaction, degree and gas information

In [65]:
# calcualte totall gas, total transfer, total receive

total_gas_used_df = data_df.groupBy('from_address').agg(_sum('gas_used (bnb)').alias('Total gas'))
total_gas_used_df=total_gas_used_df.withColumnRenamed('from_address', 'Address')

total_transfer_df = data_df.groupBy('from_address').count().withColumnRenamed('count', 'Total Transfers')
total_transfer_df=total_transfer_df.withColumnRenamed('from_address', 'Address')

total_receive_df = data_df.groupBy('to_address').count().withColumnRenamed('count', 'Total Receives')
total_receive_df=total_receive_df.withColumnRenamed('to_address', 'Address')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Intiate the Graph

Currently using graphframe, which will **not** support age weight and node weight

In [66]:
graph = GraphFrame(vertices_df, edges_df)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
# count nodes and edges
num_vertices = graph.vertices.count()
num_edges = graph.edges.count()

print("Number of nodes:", num_vertices)
print("Number of edges:", num_edges)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of nodes: 4924
Number of edges: 19154

In [68]:
# calculate in , out degree
in_degrees = graph.inDegrees
in_degrees = in_degrees.withColumnRenamed('id', 'Address')
out_degrees = graph.outDegrees
out_degrees = out_degrees.withColumnRenamed('id', 'Address')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Page Rank Calculation
Graph frame doesn't have sum of all pagerank equal to 1, so need to normalize after calculating

In [229]:
results = graph.pageRank(resetProbability=0.15, tol=0.01)
#results = graph.pageRank(resetProbability=0.5, maxIter= 10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [237]:
result_df=results.vertices.select("id", "pagerank").toDF("Address", "Reputation Score")
#result_df=result_df.drop(col('Reputation Score'))
result_df.show()#truncate=False)|

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|             Address|  Reputation Score|
+--------------------+------------------+
|0x4f129fbd6fe4c63...|0.6083179071295053|
|0x5558bf3dc2f5b0c...|0.6083179071295053|
|0x56e5272a583d3b4...|0.6083179071295053|
|0x58932ae7796b486...|0.6083179071295053|
|0xc8b9bbba82cbe9f...|0.6083179071295053|
|0xd576b5994865c94...|0.6083179071295053|
|0x0e630dc7ba2de34...|0.6083179071295053|
|0x1c537fcf80c1069...|0.6474210472661964|
|0x4e8f1cf9a9ddcad...|0.6083179071295053|
|0x54b4e4a37808b8d...|0.6083179071295053|
|0x555a56c3f71d8d1...|1.2765580580132696|
|0x67909f0fd4d7634...|0.6083179071295053|
|0x70ddb0f995d235e...|0.6083179071295053|
|0x83003d1b9d4e472...|0.6083179071295053|
|0xc0d9ae4426209cd...|0.6083179071295053|
|0xd0e1b043cb7423c...|0.6083179071295053|
|0xe8af1475e115859...|0.6083179071295053|
|0xecb0a819c0d9914...|0.6083179071295053|
|0xf764cc532f61789...|0.6083179071295053|
|0xfa97c22a03d8522...|0.8522184365084761|
+--------------------+------------

In [238]:
total_score=result_df.agg(_sum("Reputation Score")).collect()[0][0] # test total score 
total_score

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4924.000000000195

#### normalize the score 

In [239]:
result_df=result_df.withColumn('Reputation Score Scaled',col('Reputation Score')/total_score)
result_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+-----------------------+
|             Address|  Reputation Score|Reputation Score Scaled|
+--------------------+------------------+-----------------------+
|0xae9269f27437f0f...| 171.1010033163112|    0.03474837597812844|
|0x56e5272a583d3b4...|0.6083179071295053|   1.235414108711375...|
|0x555a56c3f71d8d1...|1.2765580580132696|   2.592522457378593E-4|
|0x9e2f1fdcf8ec30d...|0.7251332310642297|   1.472650753582861...|
|0x1558c9987bceaec...|0.6711716614088015|   1.363061863137235...|
|0x1c537fcf80c1069...|0.6474210472661964|   1.314827472108389...|
|0xe8af1475e115859...|0.6083179071295053|   1.235414108711375...|
|0xf764cc532f61789...|0.6083179071295053|   1.235414108711375...|
|0xfb3b893159fd02e...|0.6083179071295053|   1.235414108711375...|
|0xf89dda78a4e5e17...|0.6343580843183393|   1.288298302839793...|
|0xfa97c22a03d8522...|0.8522184365084761|   1.730744184623156...|
|0x15222740c04c23a...|0.6083179071295053|   1.235414108711375...|
|0x659edb0

In [240]:
result_df.agg(_sum("Reputation Score Scaled")).collect()[0][0] # test total score 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9999999999999521

#### scale the score to 10xe6

In [241]:
result_df=result_df.withColumn('Reputation (x10e6)',col('Reputation Score Scaled')*1000000.0)
result_df=result_df.select('Address','Reputation (x10e6)')
result_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|             Address|Reputation (x10e6)|
+--------------------+------------------+
|0x56e5272a583d3b4...|123.54141087113753|
|0xb0dd17567933fc8...| 340.2604260234871|
|0xc8b9bbba82cbe9f...|123.54141087113753|
|0xd576b5994865c94...|123.54141087113753|
|0x1c537fcf80c1069...|131.48274721083894|
|0x4e8f1cf9a9ddcad...|123.54141087113753|
|0x54b4e4a37808b8d...|123.54141087113753|
|0x555a56c3f71d8d1...|259.25224573785925|
|0x67909f0fd4d7634...|123.54141087113753|
|0xf764cc532f61789...|123.54141087113753|
|0x0cae1e6c5e8b410...|123.54141087113753|
|0x2d356e9593fb9eb...|123.54141087113753|
|0x322148a874461a5...|123.54141087113753|
|0x45909c92c0c02f5...| 121.4557322542566|
|0x6daa86ef323e5a8...|123.54141087113753|
|0x87882d214e91b2f...|226.77878328725558|
|0x88540f24dfa6440...|123.54141087113753|
|0x0225610b230e068...|123.54141087113753|
|0x142ba743cf9317e...|123.54141087113753|
|0x3ce7bcec03a4af9...|123.54141087113753|
+--------------------+------------

#### calculate in , out degree

In [242]:
in_degrees = graph.inDegrees
in_degrees = in_degrees.withColumnRenamed('id', 'Address')
out_degrees = graph.outDegrees
out_degrees = out_degrees.withColumnRenamed('id', 'Address')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Merging txn , gas, and degree information to the page rank result and sorting

In [244]:
merged_df = result_df.join(total_gas_used_df, 'Address', 'left')
merged_df = merged_df.join(total_transfer_df, 'Address', 'left')
merged_df = merged_df.join(total_receive_df, 'Address', 'left')
merged_df = merged_df.join(in_degrees, on='Address', how='left').join(out_degrees, on='Address', how='left')
merged_df = merged_df.withColumn('total_degree', merged_df['inDegree'] + merged_df['outDegree'])
merged_df = merged_df.fillna(0)
merged_df = merged_df.replace(float('-inf'), 0)
merged_df=merged_df.sort(col("Reputation (x10e6)"),ascending=False)
merged_df.show()#truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+------------+---------------+--------------+--------+---------+------------+
|             Address|Reputation (x10e6)|   Total gas|Total Transfers|Total Receives|inDegree|outDegree|total_degree|
+--------------------+------------------+------------+---------------+--------------+--------+---------+------------+
|0x75ca8f6c82df5fd...| 53313.22845002258|         0.0|              0|          1942|    1942|        0|           0|
|0xe4ae305ebe1abe6...| 43681.98293329425|         0.0|              0|           790|     790|        0|           0|
|0xae9269f27437f0f...| 34748.37597812844|         0.0|              0|          1567|    1567|        0|           0|
|0x55d398326f99059...|26884.650063964473|         0.0|              0|          1137|    1137|        0|           0|
|0xe9e7cea3dedca59...|15530.089348511083|         0.0|              0|           698|     698|        0|           0|
|0x007ea5c0ea75a8d...|12983.963025906942|         0.0|  